<a href="https://colab.research.google.com/github/edisonc57/PE-with-Python/blob/main/PE_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Material Balance Equation

*For a single-phase:*

$$
N= N_o - \Delta N + G - \phi B \Delta p
$$

In [ ]:
def remaining_reserves(N, No, delta_N, G, phi, B, delta_P):
    remaining_volume = No - delta_N + G - phi * B * delta_P
    return remaining_volume

N = 1000000        # Reservas iniciales en barriles
No = 1200000       # Reservas originales estimadas en barriles
delta_N = 200000   # Reducción debido a la extracción en barriles
G = 50000          # Aumento de reservas debido a nuevos descubrimientos en barriles
phi = 0.1          # Factor de reducción por pérdida o ineficiencia
B = 10000          # Tasa de producción en barriles por día
delta_P = 0.05     # Cambio en la presión del yacimiento

remaining_reserves(N, No, delta_N, G, phi, B, delta_P)



1049950.0

*For a two-phase:*

$$
N= N_o - \Delta N + E - D  - \phi (B_o \Delta P_o + B_g \Delta P_g )
$$

In [2]:
def two_phases_remaining_reserves(N, No, delta_N, E, D, phi, Bo, Bg,  delta_Po, delta_Pg):
    remaining_volume = No - delta_N + E - D - phi * (Bo * delta_Po + Bg * delta_Pg)
    return remaining_volume

N = 1000       # Número total de reservas estimadas inicialmente
No = 500       # Volumen inicial de reservas antes de la extracción
delta_N = 50   # Cambio en el volumen de reservas debido a la producción de petróleo
E = 200        # Volumen de inyección de fluidos (agua, gas, etc.) en el reservorio
D = 150        # Volumen de fluidos extraídos del reservorio
phi = 0.1      # Porosidad efectiva del reservorio
Bo = 1.2       # Factor volumétrico de formación del petróleo
Bg = 0.8       # Factor volumétrico de formación del gas
delta_Po = 0.05 # Cambio en la presión del reservorio de petróleo
delta_Pg = 0.03 # Cambio en la presión del reservorio de gas

two_phases_remaining_reserves (N, No, delta_N, E, D, phi, Bo, Bg,  delta_Po, delta_Pg)

499.9916

# Darcy´s Law for fluid flow